#### Show in table output gen

In [1]:
import os
import sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import pandas as pd
from constants import OW_TYPES

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
DF_BY_CASES = dict({})

base_df = None

for case in CASES:
    _df = pd.read_csv(f'../Results/{case}/results_output_gen.csv')
    _df["genInstalledCap_GW"] = _df["genInstalledCap_MW"] / 1000 # To GW
    _df = _df.drop(columns=["genInstalledCap_MW"])
    _df["genInstalledCap_GW"] = _df.groupby(['GeneratorType', 'Period'])['genInstalledCap_GW'].transform('sum')
    _df = _df.drop_duplicates(("GeneratorType", "Period"), ignore_index=True)[["GeneratorType", "Period", "genInstalledCap_GW"]]
    if case == "Base": 
        base_df = _df
    DF_BY_CASES[case] = _df

DF_BY_CASES

{'Base':         GeneratorType     Period  genInstalledCap_GW
 0    Liginiteexisting  2020-2025           61.315000
 1    Liginiteexisting  2025-2030           37.732308
 2    Liginiteexisting  2030-2035           25.351394
 3    Liginiteexisting  2035-2040           17.687019
 4    Liginiteexisting  2040-2045           12.263000
 ..                ...        ...                 ...
 219           Nuclear  2035-2040          191.571583
 220           Nuclear  2040-2045          185.578799
 221           Nuclear  2045-2050          242.896926
 222           Nuclear  2050-2055          226.092070
 223           Nuclear  2055-2060          226.092070
 
 [224 rows x 3 columns],
 'NOgrid':         GeneratorType     Period  genInstalledCap_GW
 0    Liginiteexisting  2020-2025           61.315000
 1    Liginiteexisting  2025-2030           37.732308
 2    Liginiteexisting  2030-2035           25.351394
 3    Liginiteexisting  2035-2040           17.687019
 4    Liginiteexisting  2040-2045    

Calculate diff to base

In [2]:
base_df = base_df.rename(columns={"genInstalledCap_GW": "base_genInstalledCap_GW"})
for case, _df in DF_BY_CASES.items():
    new_df = _df.merge(base_df, on=["GeneratorType", "Period"], how="left")
    new_df["Diff to base"] = round(new_df["genInstalledCap_GW"] - new_df["base_genInstalledCap_GW"])
    new_df["Diff to base %"] = round((new_df["genInstalledCap_GW"]/new_df["base_genInstalledCap_GW"] - 1) * 100, 1)
    new_df["genInstalledCap_GW"] = round(new_df["genInstalledCap_GW"])
    DF_BY_CASES[case] = new_df

Filter by case and period

In [4]:
FILTER_CASE = "NOgrid"
FILTER_PERIOD = "2045-2050"

filtered_df = DF_BY_CASES[FILTER_CASE]
filtered_df = filtered_df[filtered_df["Period"] == FILTER_PERIOD].sort_values("genInstalledCap_GW", ascending=False).reset_index(drop=True)
print(f"Total offshore wind prod: {filtered_df[filtered_df["GeneratorType"].isin(OW_TYPES)]["genInstalledCap_GW"].sum()}GW")
filtered_df

Total offshore wind prod: 234.0GW


,GeneratorType,Period,genInstalledCap_GW,base_genInstalledCap_GW,Diff to base,Diff to base %
0,Windonshore,2045-2050,1729.0,1731.044410,-2.0,-0.1
1,Solar,2045-2050,1528.0,1508.851809,19.0,1.2
2,Nuclear,2045-2050,282.0,242.896926,39.0,16.1
3,GasCCGT,2045-2050,243.0,241.390379,2.0,0.7
4,Bio,2045-2050,243.0,253.563914,-10.0,-4.1
5,Windoffshoregrounded,2045-2050,215.0,251.656969,-37.0,-14.6
6,GasOCGT,2045-2050,166.0,169.610585,-4.0,-2.2
7,Hydrorun-of-the-river,2045-2050,123.0,125.703597,-2.0,-2.0
8,Hydroregulated,2045-2050,103.0,106.054762,-3.0,-3.2
9,Lignite,2045-2050,29.0,28.746930,-0.0,-0.4
